In [1]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os
import pandas as pd
import re

In [2]:
load_dotenv()

# Read variables
db_host = os.getenv("MYSQL_HOSTNAME")
db_database = os.getenv("MYSQL_DATABASE")
db_user = os.getenv("SELECT_USER")
db_password = os.getenv("SELECT_PW")

# db_user = os.getenv("ADMIN_USER")
# db_password = os.getenv("ADMIN_PW")

remove_pkgs = os.getenv("PACKAGES_TO_EXCLUDE").split(",")
cols_to_keep= os.getenv("COLUMNS_TO_KEEP").split(",")


In [4]:
try:
    connection = mysql.connector.connect(host=db_host,
                                            database=db_database,
                                            user=db_user,
                                            password=db_password)
    if connection.is_connected():
        query = "SELECT * FROM screentext";
        df = pd.read_sql(query, connection)

        # Export DataFrame to CSV
        df.to_csv("data/screentext_export.csv", index=False)
        print("Data exported successfully!")
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

C:\Users\edita\AppData\Local\Temp\ipykernel_15336\735628796.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Data exported successfully!
MySQL connection is closed


In [5]:
# Read screen_text_export.csv and remove rows with package_name in remove_pkgs<<
df = pd.read_csv("screentext_export.csv")
df = df[~df["package_name"].isin(remove_pkgs)]
# Remove columns not in cols_to_keep
df = df[cols_to_keep]

In [6]:
# Function to parse screentext into phrases
def parse_phrases(text):
    pattern = re.compile(r'(.*?)\*\*\*Rect\((-?\d+), (-?\d+) - (-?\d+), (-?\d+)\)\|\|', re.DOTALL)
    return [match[0].strip() for match in pattern.findall(text)]


In [8]:
from collections import deque


def convert_timestamps(df, timestamp_column='timestamp'):
    df_converted = df.copy()
    df_converted[timestamp_column] = pd.to_datetime(df_converted[timestamp_column], unit='ms')
    df_converted = df_converted.groupby(["device_id", pd.Grouper(key='timestamp', freq='2s')]).first().reset_index()
    return df_converted

df_converted = convert_timestamps(df)
df_converted.to_csv("data/frequency.csv", index=False)

device_groups = df_converted.groupby("device_id")
filtered_rows = []

# this removes duplicate phrases within a 5-minute window, if there are some duplicates phrases in a row the phrases will be removed, if all phrases in a row are duplicated, remove the row
for _, device_df in device_groups:
    seen_phrases = deque()  # Stores (timestamp, phrases) tuples
    kept_phrases = set()  # Active unique phrases within 5-minute window
    
    for index, row in device_df.iterrows():
        phrases = set(parse_phrases(row["text"]))
        print(phrases)
        timestamp = row["timestamp"]
        
        # Remove old phrases outside the 5-minute window
        while seen_phrases and (timestamp - seen_phrases[0][0]).total_seconds() > 300:
            old_timestamp, old_phrases = seen_phrases.popleft()
            kept_phrases.difference_update(old_phrases)
        
        new_phrases = phrases - kept_phrases  
        if not new_phrases:  
            continue
                
        seen_phrases.append((timestamp, new_phrases))
        kept_phrases.update(new_phrases)
        
        row["text"] = ",".join(new_phrases) 
        filtered_rows.append(row)

# Create new dataframe from filtered rows
df_filtered = pd.DataFrame(filtered_rows)
# Save to filtered_frequency_5s.csv
df_filtered.to_csv("data/screentext_removed_duplicates.csv", index=False)


{'Press and hold Volume up and down keys for 3 seconds', 'AWARE-Light', "AWARE is an Android's framework dedicated to infer, log and share mobile context information between mobile applications. Visit http://awareframework.com for details.", 'AWARE-Light shortcut'}
{'bob', 'Device control', 'AWARE', '16:07', 'Data collection active', 'Sun, 2 Mar', 'Media output', 'Display brightness'}
{'(7.44 MB) amazonaws.com', 'File downloaded', 'Open'}
{'BIO Maca v prahu, 500 g - FutuNatura.si', 'aware-light.org/downloads/', 'Downloads – AWARE-Light', 'zelda hyrule bass - Google Search', 'botw heart container or stamina vessel - Google Search'}
{'Instagram', '🦉: skoVir : vir skovikov :🦉', 'Search or type URL', 'File downloaded', 'Rail Vision Ltd. (RVSN) Stock Price, News, Quote & History - Yahoo Finance', 'Open', 'Amazon.de: Low Prices in Electronics, Books, Sports Equipment & more', 'Discover', 'gmail', '"Please make sure you are logged on to steam" :: Palia General Discussions', 'ai stars icon', '